In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from time import time
from torch.optim.lr_scheduler import ExponentialLR
from torch.cuda.amp import autocast as autocast

定义模型

In [ ]:
# 定义神经网络  
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # output: 64 x 16 x 16
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # output: 128 x 8 x 8
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # output: 256 x 4 x 4
        )
        # 全连接层
        self.dense = nn.Sequential(
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(-1, 256*4*4)
        x = self.dense(x)
        return x

In [ ]:
%%capture
model = Net()
model.cuda()

In [ ]:
train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, hue=0.2)
    ])

test_transform = transforms.Compose([
        transforms.ToTensor()
    ])

In [ ]:
train_data = datasets.CIFAR10("./data", train=True, download=True, transform=train_transform)
test_data = datasets.CIFAR10("./data", train=False, download=True, transform=test_transform)
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=True)

images, labels = next(iter(test_loader))

In [ ]:
def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)#求每行的最大就是最有可能的类别
    num_correct = (pred_label == label).sum().float()
    return num_correct / total

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
scheduler = ExponentialLR(optimizer, gamma=0.9)

In [ ]:
for epoch in range(20):
    t1 = time()
    train_loss = 0
    train_acc = 0
    model = model.train()
    for i, (images, labels) in enumerate(train_loader):# im,label为一批数据，也就是64个样本
        images, labels = images.cuda(), labels.cuda()
        optimizer.zero_grad()
        #with autocast(): 
        output = model(images)
        # loss = F.cross_entropy(output, labels)
        loss = criterion(output ,labels)
        # 反向传播
        loss.backward()
        optimizer.step()
        train_loss += loss.data.float()
        train_acc += get_acc(output,labels)    
    scheduler.step()
    t2 = time()
    print("Spend Time: %s" % (t2 - t1))
    print("Train Accuracy: %s" % (train_acc/len(train_loader)).cpu())

In [4]:
# 使用tensor.dtype查看数据类型
a_float32 = torch.rand((8, 8), device="cuda")
b_float32 = torch.rand((8, 8), device="cuda")
c_float32 = torch.rand((8, 8), device="cuda")
d_float32 = torch.rand((8, 8), device="cuda")

with autocast():
    e_float16 = torch.mm(a_float32, b_float32)
    f_float16 = torch.mm(d_float32, e_float16)
g_float32 = torch.mm(d_float32, f_float16)#报错
# g_float32 = torch.mm(d_float32, f_float16.float())

RuntimeError: expected scalar type Float but found Half

In [3]:
print(a_float32.dtype)

torch.float32
